In [1]:
import pandas as pd
import openai
import csv
import time
import os
import regex as re
import math
import random

import openai

pd.set_option('display.max_colwidth', None)
file_path = "D:/api_key.txt"
with open(file_path, 'r') as file:
    api_key = file.read()

openai.api_key = (api_key)

In [2]:
def label_data(original):
    message_history = []
    #system_input_long = "Correct and improve Dutch text. Assume there is unprovided context. If the input can be improved upon, provide the corrected Dutch text and nothing else. If the input is not grammatically/semantically correct and you can't fix the text, respond with 'dzisis'. If the input is completely correct, respond with 'dzizizi'. Otherwise give the corrected version. Never provide any additional information"
    system_input_long = "Verbeter nederlandse medische tekst. Als de input verbeterd kan worden, geef de verbeterde versie van de tekst en niets anders. Als de input niet verbeterd kan worden, reageer met: 'dzizizi'. Als de input onbegrijpelijk is reageer dan met 'dzisis'. Geef geen extra informatie."
    message_history.append({'role' : "system", "content": system_input_long})
    message_history.append({"role": "user", 'content' : original})

    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',messages=message_history, temperature=0.3,top_p=0.9)
    reply_content = completion.choices[0].message.content
    return reply_content

In [3]:
def generate_labels_faulty_data(df_generated, df_example_1, df_example_2, output_file_path, generation_limit):
    # check if output file already exists
    if os.path.exists(output_file_path):
        # if it does, read the last row that was successfully processed
        with open(output_file_path, 'r', encoding='utf-8') as output_file:
            reader = csv.reader(output_file)
            num_filled_lines = 0
            for row in reader:
                if any(row):
                    num_filled_lines += 1
            last_row_processed = num_filled_lines - 1
    else:
        # if it doesn't, set last_row_processed to 0
        last_row_processed = 0

    # open input and output files
    with open(output_file_path, 'a', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)

        #If initializing a new document, skip the header and rewrite the new format.
        if last_row_processed == 0:
            writer.writerow(['Original', 'Generated', 'Example_1', 'Example_2', 'Understood', 'Correct'])
        number_generated = 0
        for index in range(last_row_processed, len(df_generated)):
            original = df_generated.iloc[index]
            example_1 = df_example_1.iloc[index]
            example_2 = df_example_2.iloc[index]
            #original, example_1, example_2 = row['generated'], row['example_1'], row['example2'] 
            time.sleep(20)
            try:
                reply_content = label_data(original)
                print(reply_content)
            except:
                # if API call fails, wait for 5 seconds and retry
                print('failed... retrying in 5 seconds')
                time.sleep(5)
                #Longer prompt costs more.
                reply_content = label_data(original)
                print(reply_content)

            correct_sentence = bool(re.search(r'dzizizi', reply_content.lower()))
            misunderstood_sentence = bool(re.search(r'dzisis', reply_content.lower()))

            understood = True
            correct = False
            if misunderstood_sentence == True:
                reply_content = original
                understood = False

            if correct_sentence == True:
                reply_content = original
                correct = True

            writer.writerow([original, reply_content, example_1, example_2, understood, correct])
            number_generated += 1

            # flush output file buffer to ensure data is saved
            output_file.flush()

            # print progress
            #print(f"Processed row {last_row_processed}")
            print('Rows Processed:', number_generated)
            # break out of loop if num_rows is reached
            if generation_limit is not None and number_generated >= generation_limit:
                print('Limit reached')
                break

In [4]:
df = pd.read_csv('E:/Data_exploration/GitHub/paraphraser_code/data/generated/generated_incorrect_medical.csv')
generate_labels_faulty_data(df['generated'], df['example_1'], df['example2'], output_file_path='E:/Data_exploration/GitHub/paraphraser_code/data/generated/generated_annotated_incorrect_medical.csv', generation_limit=1000)

dzizizi
Rows Processed: 1
Gisteren klaagde patiënte mevrouw Janssen over zeer zware buikpijn en had ze diarree. We hebben haar medicatie gegeven en haar gevraagd om veel water te drinken. Ze heeft zich daarna rustig gehouden op bed en is niet naar het toilet geweest. Vanochtend hebben we haar weer verschoond en geholpen naar het toilet te gaan, waarna ze zich iets beter voelde. Vervolgens hebben we haar ontbijt gegeven, maar ze had geen zin om te eten. We houden haar de komende dagen goed in de gaten en blijven medicatie toedienen.
Rows Processed: 2
Vandaag had ik een afspraak met de patiënt, meneer Janssen. Hij kwam binnen met klachten over zijn benen. Ik heb hem gevraagd om op het bed te gaan liggen. Vervolgens heb ik zijn temperatuur opgemeten, deze was echter iets verhoogd. Daarna heb ik zijn bloeddruk en hartslag gemeten, deze waren goed. Ik heb hem een paar pijnstillers gegeven en hem geadviseerd om meer rust te nemen. Verder heb ik hem een folder gegeven over gezonde voeding, om

In [11]:
new_df = pd.read_csv('E:/Data_exploration/GitHub/paraphraser_code/data/generated/generated_annotated_incorrect_medical.csv', encoding='utf-8', header=0)
new_df.head()

,Original,Generated,example_1,example_2,Understood,Correct
0,Meneer kom lated aan in het ziekehuiss vanmorg...,Meneer is vanochtend laat aangekomen in het zi...,Mw lag er rustig bij. Controle tena. geen urin...,Lichamelijk welbevinden: Mevrouw haar vistal e...,True,False
1,Meneer klagt dat hij pijn voelt in ze borst. H...,Meneer klaagt dat hij pijn voelt in zijn borst...,Mw lag er rustig bij. Controle tena. geen urin...,Lichamelijk welbevinden: Mevrouw haar vistal e...,True,False
2,Ik hebb Meneer van der Veen geassisteert met h...,Ik heb meneer Van der Veen geholpen bij het na...,Mw lag er rustig bij. Controle tena. geen urin...,Lichamelijk welbevinden: Mevrouw haar vistal e...,True,False
